In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
import  matplotlib.pyplot  as plt
import gensim
import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline

In [6]:
nlp = spacy.load("en_core_web_sm")

In [9]:
from rcare.readutils import DataFrameUtils as Utils

In [11]:
DATA_PATH = 'data/lexpredict-contraxsuite-samples/credit_card_agreements/citi/'
df_files = Utils.read_data(path = DATA_PATH)

2018-08-03 10:26:37,875 : INFO : data/lexpredict-contraxsuite-samples/credit_card_agreements/citi/*********************
2018-08-03 10:26:37,876 : INFO : ['SearsCharge_PLUS_Account.pdf.txt', 'Citi_ThankYou_Premier_Card.pdf.txt', 'Citi_Forward_Card.pdf.txt', 'Expedia+_Voyager_Card.pdf.txt', 'Tractor_Supply_Company_Personal_Credit_Card.pdf.txt', 'Citi_AAdvantage_Platinum_Select_Card.pdf.txt', 'My_BestBuy_Visa.pdf.txt', 'Piercing_Pagoda_Credit_Card.pdf.txt', 'Sears_Home_Improvement_Account.pdf.txt', 'Sears_Card.pdf.txt', 'Citi_Gold.pdf.txt', 'Citi_Double_Cash_Card.pdf.txt', 'Citi_AAdvantage_Bronze_World_MasterCard.pdf.txt', 'Citi_Dividend_American_Express_Card.pdf.txt', 'Citi_Card.pdf.txt', 'Citi_ThankYou_Preferred_Card.pdf.txt', "The_Children's_Place_Card.pdf.txt", 'Hilton_HH_Honors_Visa_Gold_Card.pdf.txt', 'Citi_Prestige_Card.pdf.txt', 'Merchant_Tire_Credit_Card.pdf.txt', 'Sunoco_Rewards_Card.pdf.txt', 'ExxonMobil_Preferred_Smart_Card.pdf.txt', 'Pacific_Sales_Credit_Card.pdf.txt', 'Citig

2018-08-03 10:26:38,156 : INFO : Appending to dataframe Smith_Barney_Platinum_Select_Visa.pdf.txt
2018-08-03 10:26:38,161 : INFO : Reading file Classic_Silver_Secured_Card.pdf.txt
2018-08-03 10:26:38,164 : INFO : Appending to dataframe Classic_Silver_Secured_Card.pdf.txt
2018-08-03 10:26:38,170 : INFO : Reading file Citi_Dividend_World_MasterCard.pdf.txt
2018-08-03 10:26:38,173 : INFO : Appending to dataframe Citi_Dividend_World_MasterCard.pdf.txt
2018-08-03 10:26:38,177 : INFO : Reading file Hilton_HHonors_Visa_Signature_Card.pdf.txt
2018-08-03 10:26:38,181 : INFO : Appending to dataframe Hilton_HHonors_Visa_Signature_Card.pdf.txt
2018-08-03 10:26:38,187 : INFO : Reading file CITGO_Plus_Credit_Card.pdf.txt
2018-08-03 10:26:38,191 : INFO : Appending to dataframe CITGO_Plus_Credit_Card.pdf.txt
2018-08-03 10:26:38,197 : INFO : Reading file Citi_Simplicity_Card.pdf.txt
2018-08-03 10:26:38,206 : INFO : Appending to dataframe Citi_Simplicity_Card.pdf.txt
2018-08-03 10:26:38,212 : INFO : Rea

2018-08-03 10:26:38,587 : INFO : Reading file Citi_AAdvantage_Platinum_Select_American_Express.pdf.txt
2018-08-03 10:26:38,591 : INFO : Appending to dataframe Citi_AAdvantage_Platinum_Select_American_Express.pdf.txt
2018-08-03 10:26:38,596 : INFO : Reading file Citi_Health_Account_Card.pdf.txt
2018-08-03 10:26:38,600 : INFO : Appending to dataframe Citi_Health_Account_Card.pdf.txt
2018-08-03 10:26:38,606 : INFO : Reading file ExxonMobil_Personal_Card.pdf.txt
2018-08-03 10:26:38,609 : INFO : Appending to dataframe ExxonMobil_Personal_Card.pdf.txt
2018-08-03 10:26:38,615 : INFO : Reading file data done..


In [78]:
from nltk.corpus import stopwords
stopwordlist = stopwords.words('english')

for idx, stopword in enumerate(stopwordlist):
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True
    #print(idx, lexeme.text, stopword)

In [70]:
#df_files

# we add some words to the stop word list# we ad 
docs, article = [], []
for index, row in df_files[:40].iterrows():    
    doc = nlp(row['data'])
    for sent in doc.sents:
        for w in sent:
            # if it's not a stop word or punctuation mark, add it to our article!
            if w.text not in ('\n', '$','\n\n' , '\n\n\n', '\n\n\n\n'  ) and w.lemma_ != '-PRON-'\
            and not w.is_stop and not w.is_punct and not w.like_num:
                # we add the lematized version of the word
                article.append(w.lemma_)
        # if it's a new line, it means we're onto our next document
        docs.append(article)
        article = []
    
    

In [72]:
#len(docs)
bigram  =  gensim.models.Phrases(docs)

2018-08-03 13:32:30,657 : INFO : collecting all words and their counts
2018-08-03 13:32:30,660 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-08-03 13:32:30,933 : INFO : PROGRESS: at sentence #10000, processed 71960 words and 4378 word types
2018-08-03 13:32:31,132 : INFO : collected 5619 word types from a corpus of 123556 words (unigram + bigrams) and 17222 sentences
2018-08-03 13:32:31,134 : INFO : using 5619 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [73]:
docs = [bigram[line] for line in docs]

In [74]:
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

2018-08-03 13:32:54,200 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-08-03 13:32:54,378 : INFO : adding document #10000 to Dictionary(1136 unique tokens: ['agreement', 'card', 'scpl-0811', 'contract', 'this']...)
2018-08-03 13:32:54,491 : INFO : built Dictionary(1457 unique tokens: ['agreement', 'card', 'scpl-0811', 'contract', 'this']...) from 17222 documents (total 108044 corpus positions)


In [79]:
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary)

2018-08-03 14:19:34,300 : INFO : using symmetric alpha at 0.2
2018-08-03 14:19:34,303 : INFO : using symmetric eta at 0.2
2018-08-03 14:19:34,305 : INFO : using serial LDA version on this node
2018-08-03 14:19:34,311 : INFO : running online (single-pass) LDA training, 5 topics, 1 passes over the supplied corpus of 17222 documents, updating model once every 2000 documents, evaluating perplexity every 17222 documents, iterating 50x with a convergence threshold of 0.001000
2018-08-03 14:19:34,312 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-08-03 14:19:34,314 : INFO : PROGRESS: pass 0, at document #2000/17222
2018-08-03 14:19:35,749 : INFO : merging changes from 2000 documents into a model of 17222 documents
2018-08-03 14:19:35,754 : INFO : topic #0 (0.200): 0.053*"balance" + 0.043*"interest" + 0.041*"payment" + 0.035*"charge" + 0.032*"fee" + 0.021*"account" + 0.018*"apply" + 0.017*"the" + 0.016*"

2018-08-03 14:19:39,518 : INFO : topic #4 (0.200): 0.042*"credit" + 0.028*"rate" + 0.025*"cash" + 0.023*"card" + 0.023*"agreement" + 0.023*"balance" + 0.022*"limit" + 0.022*"use" + 0.021*"if" + 0.021*"advance"
2018-08-03 14:19:39,519 : INFO : topic diff=0.155717, rho=0.408248
2018-08-03 14:19:39,521 : INFO : PROGRESS: pass 0, at document #14000/17222
2018-08-03 14:19:40,166 : INFO : merging changes from 2000 documents into a model of 17222 documents
2018-08-03 14:19:40,170 : INFO : topic #0 (0.200): 0.076*"balance" + 0.053*"payment" + 0.051*"interest" + 0.047*"charge" + 0.029*"fee" + 0.024*"add" + 0.023*"apply" + 0.021*"account" + 0.019*"minimum" + 0.019*"promotional"
2018-08-03 14:19:40,172 : INFO : topic #1 (0.200): 0.046*"billing" + 0.037*"fee" + 0.035*"payment" + 0.034*"the" + 0.033*"pay" + 0.032*"if" + 0.030*"statement" + 0.027*"period" + 0.022*"day" + 0.015*"arbitration"
2018-08-03 14:19:40,174 : INFO : topic #2 (0.200): 0.048*"account" + 0.041*"law" + 0.036*"â¢" + 0.030*"claim"

In [80]:
ldamodel.show_topics()

[(0,
  '0.079*"balance" + 0.055*"payment" + 0.051*"interest" + 0.046*"charge" + 0.028*"fee" + 0.023*"add" + 0.021*"apply" + 0.021*"account" + 0.020*"promotional" + 0.019*"minimum"'),
 (1,
  '0.045*"billing" + 0.038*"fee" + 0.035*"payment" + 0.034*"the" + 0.033*"pay" + 0.032*"if" + 0.030*"statement" + 0.025*"period" + 0.021*"day" + 0.015*"arbitration"'),
 (2,
  '0.048*"account" + 0.043*"law" + 0.034*"â\x80¢" + 0.031*"claim" + 0.028*"party" + 0.023*"arbitration" + 0.019*"include" + 0.016*"non" + 0.015*"this" + 0.014*"permit"'),
 (3,
  '0.090*"arbitration" + 0.034*"claim" + 0.025*"account" + 0.020*"information" + 0.020*"card" + 0.017*"follow" + 0.015*"use" + 0.014*"rule" + 0.013*"claims" + 0.012*"procedure"'),
 (4,
  '0.039*"credit" + 0.029*"rate" + 0.027*"cash" + 0.026*"card" + 0.024*"agreement" + 0.023*"use" + 0.022*"advance" + 0.022*"limit" + 0.021*"if" + 0.020*"account"')]

In [81]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.215201 -0.124181       1        1  27.067957
1      0.170563  0.025223       2        1  20.452810
4      0.036021  0.194832       3        1  20.070349
2     -0.190665 -0.166972       4        1  16.664734
3     -0.231119  0.071098       5        1  15.744157, topic_info=     Category         Freq            Term        Total  loglift  logprob
term                                                                     
408   Default  2425.000000     arbitration  2425.000000  30.0000  30.0000
70    Default  2982.000000         balance  2982.000000  29.0000  29.0000
138   Default  1619.000000        interest  1619.000000  28.0000  28.0000
57    Default  1435.000000          charge  1435.000000  27.0000  27.0000
134   Default  2512.000000         payment  2512.000000  26.0000  26.0000
437   Default  1153.000000           claim  1153.000000  25.0000  25.0000
137   Default  1723.000000             fee  1723.000000  24.0000  24.0000
110   Default   973.000000             law   973.000000  23.0000  23.0000
99    Default  1329.000000         billing  1329.000000  22.0000  22.0000
220   Default   887.000000             â¢   887.000000  21.0000  21.0000
5     Default  2315.000000         account  2315.000000  20.0000  20.0000
1     Default   948.000000            card   948.000000  19.0000  19.0000
100   Default   990.000000       statement   990.000000  18.0000  18.0000
26    Default  1529.000000          credit  1529.000000  17.0000  17.0000
181   Default   784.000000          period   784.000000  16.0000  16.0000
140   Default   561.000000         minimum   561.000000  15.0000  15.0000
170   Default   595.000000     promotional   595.000000  14.0000  14.0000
319   Default   970.000000            cash   970.000000  13.0000  13.0000
115   Default   414.000000          change   414.000000  12.0000  12.0000
94    Default   592.000000           limit   592.000000  11.0000  11.0000
389   Default   916.000000           party   916.000000  10.0000  10.0000
7     Default  1008.000000             use  1008.000000   9.0000   9.0000
139   Default   420.000000             due   420.000000   8.0000   8.0000
0     Default   967.000000       agreement   967.000000   7.0000   7.0000
132   Default   889.000000             add   889.000000   6.0000   6.0000
473   Default   281.000000             non   281.000000   5.0000   5.0000
369   Default   791.000000         advance   791.000000   4.0000   4.0000
8     Default   662.000000     information   662.000000   3.0000   3.0000
38    Default  1628.000000             pay  1628.000000   2.0000   2.0000
121   Default   773.000000             new   773.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
748    Topic5    43.247040             atm    44.464539   1.8209  -5.9747
437    Topic5   580.508606           claim  1153.702637   1.1619  -3.3777
516    Topic5   241.157471            rule   399.305023   1.3444  -4.2561
179    Topic5    84.578560             any   105.526573   1.6274  -5.3039
8      Topic5   346.719513     information   662.958496   1.2005  -3.8931
443    Topic5   163.471542            what   252.888214   1.4124  -4.6450
367    Topic5   195.276535       provision   323.927277   1.3426  -4.4672
243    Topic5    98.997292         current   137.445419   1.5206  -5.1465
23     Topic5   135.578094  authorize_user   234.980560   1.2987  -4.8320
1      Topic5   333.973297            card   948.410645   0.8050  -3.9305
456    Topic5   226.072906          claims   581.242981   0.9044  -4.3207
420    Topic5   120.673302        initiate   220.353119   1.2466  -4.9485
206    Topic5    84.411499              in   129.710358   1.4191  -5.3059
424    Topic5   189.243317      arbitrator   523.672485   0.8309  -4.4986
55     Topic5   206.694260         subject   613.733154   0.7604  -4.4103
5      Topic5   432.388580 

In [42]:
#corpus

In [44]:
from nltk.corpus import stopwords
stopwordlist = stopwords.words('english')

In [49]:
'if' in stopwordlist

True

In [58]:
len(docs[6])

3478

In [82]:
dictionary.__dict__

{'token2id': {'agreement': 0,
  'card': 1,
  'scpl-0811': 2,
  'contract': 3,
  'this': 4,
  'account': 5,
  'govern': 6,
  'use': 7,
  'information': 8,
  'supplemental_pricing': 9,
  'the': 10,
  'â\x80\x9csupplementâ\x80\x9d': 11,
  'include': 12,
  'please_read': 13,
  'supplement_carefully': 14,
  'keep_record': 15,
  'definitions': 16,
  'mean': 17,
  'relationship_establish': 18,
  'annual_percentage': 19,
  'apr': 20,
  'rate': 21,
  'allow': 22,
  'authorize_user': 23,
  'person': 24,
  'access_device': 25,
  'credit': 26,
  'number': 27,
  'citibank_n.a.': 28,
  'issuer': 29,
  'falls_sd': 30,
  'locate_sioux': 31,
  'apply': 32,
  'open': 33,
  'responsible_comply': 34,
  'accordance': 35,
  'agree': 36,
  'amount': 37,
  'pay': 38,
  'authorize': 39,
  'bind': 40,
  'close': 41,
  'day': 42,
  'receive': 43,
  'lawful': 44,
  'transaction': 45,
  'authorized_users': 46,
  'additional': 47,
  'request': 48,
  'agent': 49,
  'each': 50,
  'extent': 51,
  'impose': 52,
  'limi